In [120]:
import datetime
import random

import polars as pl
import plotly.express as px

In [161]:
def convert_modeldate_to_date(df: pl.DataFrame, col: str = "modeldate") -> pl.DataFrame:
    return df.with_columns(
        pl.col(col).str.split_exact("/", 2).struct.rename_fields(["month", "day", "year"]).alias("date_parts")
    ).with_columns(
        pl.concat_str(
            pl.lit("2024"),
            pl.col("date_parts").struct.field("month").str.pad_start(2, "0"),
            pl.col("date_parts").struct.field("day").str.pad_start(2, "0"),
            separator="-",
        ).str.to_date(r"%Y-%m-%d").alias(col)
    ).drop("date_parts")

In [153]:
df = pl.read_excel("State_Topline.xlsx", engine="xlsx2csv").pipe(convert_modeldate_to_date)
df

modeldate,candidate_inc,candidate_chal,candidate_3rd,state,tipping,vpi,winstate_inc,winstate_chal,winstate_3rd,voteshare_inc,voteshare_chal,voteshare_3rd,voteshare_other,voteshare_inc_hi,voteshare_chal_hi,voteshare_3rd_hi,voteshare_other_hi,voteshare_inc_lo,voteshare_chal_lo,voteshare_3rd_lo,voteshare_other_lo,margin,margin_hi,margin_lo,win_ec_if_win_state_inc,win_ec_if_win_state_chal,win_state_if_win_ec_inc,win_state_if_win_ec_chal,state_turnout,state_turnout_hi,state_turnout_lo
date,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64
2024-07-29,"""Harris""","""Trump""","""Kennedy""","""Alabama""",0.00045,0.031036,0.005025,0.993775,0.0012,33.84494,61.96524,3.322867,0.86696,39.925079,68.558296,8.50839,1.684567,27.75782,55.107841,0.141859,0.245088,-28.1203,-16.48624,-39.720169,0.865672,0.625393,0.011702,0.998273,2105602,2307568,1908547
2024-07-29,"""Harris""","""Trump""","""Kennedy""","""Alaska""",0.002163,0.972386,0.16275,0.82445,0.0128,39.092072,53.119499,5.483664,2.304762,49.434879,63.874722,13.94918,4.40118,28.626551,41.900421,0.238457,0.698577,-14.02743,4.914183,-32.971489,0.697081,0.687428,0.305199,0.910332,321775,350954,293139
2024-07-29,"""Harris""","""Trump""","""Kennedy""","""Arizona""",0.03975,1.770612,0.243725,0.756225,0.00005,45.644039,50.611881,2.249342,1.494744,50.620201,55.63657,6.265196,2.842846,40.641079,45.5224,0.0,0.432838,-4.967846,4.262665,-14.23797,0.962868,0.812457,0.631313,0.986869,3241968,3523168,2963302
2024-07-29,"""Harris""","""Trump""","""Kennedy""","""Arkansas""",0.000137,0.018698,0.004,0.99355,0.00245,32.008751,62.10326,4.269988,1.618003,38.454151,69.33139,10.72017,3.094144,25.545059,54.42057,0.201923,0.481643,-30.0945,-17.831989,-42.31852,0.8875,0.624981,0.00955,0.99739,1109428,1217163,1002799
2024-07-29,"""Harris""","""Trump""","""Kennedy""","""California""",0.000575,0.005329,0.99495,0.00365,0.0014,59.913139,34.079079,4.469375,1.538404,66.435867,39.97686,10.44721,2.84859,53.12508,28.206051,0.289167,0.500484,25.83407,36.740089,14.8892,0.373361,0.965753,0.999327,0.005662,15609755,17081524,14168762
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-11-04,"""Harris""","""Trump""",null,"""Virginia""",0.0067,0.238091,0.93895,0.06105,null,52.587502,46.312618,null,1.099874,55.187321,48.9137,null,2.019382,49.980068,43.714741,null,0.374822,6.274879,11.39111,1.176607,0.522605,0.977477,0.997358,0.11845,4389741,4773191,4014911
2024-11-04,"""Harris""","""Trump""",null,"""Washington""",0.00075,0.028393,0.9965,0.0035,null,57.17572,40.474411,null,2.349868,60.649399,43.850639,null,4.251854,53.678558,37.094898,null,0.85833,16.701309,23.31216,10.11366,0.492925,0.771429,0.998374,0.005359,4044898,4399740,3695181
2024-11-04,"""Harris""","""Trump""",null,"""West Virginia""",0.000075,0.016221,0.002725,0.997275,null,31.26486,67.837631,null,0.897509,37.565868,74.07132,null,1.67585,25.051611,61.531849,null,0.298389,-36.572769,-24.01425,-48.975929,0.715596,0.504399,0.003963,0.998462,740641,818289,665209


In [297]:
SWING_STATES = [
    "Arizona",
    "Florida",
    "Georgia",
    # "Iowa",
    "Michigan",
    # "Minnesota",
    "North Carolina",
    "Nevada",
    # "Ohio",
    "Pennsylvania",
    # "Texas",
    "Wisconsin",
    # "New Mexico",
]


In [298]:
COHORT_1 = [
    "Georgia",
    # "Iowa",
    "Michigan",
    # "Minnesota",
    # "Ohio",
    "Pennsylvania",
    "Wisconsin",
]

RANDOM_SAMPLE = False
if RANDOM_SAMPLE:
    COHORT_1 = random.sample(SWING_STATES, len(SWING_STATES) // 2)

COHORT_2 = sorted(set(SWING_STATES) - set(COHORT_1))

print(
    f"Cohort 1: {COHORT_1}\nCohort 2: {COHORT_2}\n"
)


Cohort 1: ['Georgia', 'Michigan', 'Pennsylvania', 'Wisconsin']
Cohort 2: ['Arizona', 'Florida', 'Nevada', 'North Carolina']



In [300]:
WINDOW_SIZE = 30
DIFF_LAG = 3

df_cohort = df.filter(
    pl.col("modeldate") >= datetime.date(2024, 8, 25),
    pl.col("state").is_in(SWING_STATES),
).with_columns(
    pl.col("state").is_in(COHORT_1).replace_strict({True: "oct_28_mover", False: "counterfactual"}, return_dtype=pl.String).alias("swing_state_cohort")
).group_by("modeldate", "swing_state_cohort").agg(
    pl.col("voteshare_inc").mean().alias("voteshare_inc_group")
).sort("modeldate", "swing_state_cohort").pivot(
    index="modeldate",
    on="swing_state_cohort",
    values="voteshare_inc_group",
).with_columns(
    pl.col("oct_28_mover").diff(DIFF_LAG).alias("oct_28_mover_diff"),
    pl.col("counterfactual").diff(DIFF_LAG).alias("counterfactual_diff"),
).with_columns(
    pl.rolling_corr("oct_28_mover", "counterfactual", window_size=WINDOW_SIZE).alias("corr"),
    pl.rolling_corr("oct_28_mover_diff", "counterfactual_diff", window_size=WINDOW_SIZE).alias("corr_diff"),
)
df_cohort

modeldate,counterfactual,oct_28_mover,oct_28_mover_diff,counterfactual_diff,corr,corr_diff
date,f64,f64,f64,f64,f64,f64
2024-08-25,48.54493,49.798393,null,null,null,null
2024-08-26,48.562725,49.806725,null,null,null,null
2024-08-27,48.44777,49.727636,null,null,null,null
2024-08-28,48.40905,49.657818,-0.140575,-0.13588,null,null
2024-08-29,48.233185,49.466167,-0.340557,-0.32954,null,null
…,…,…,…,…,…,…
2024-10-31,48.15078,49.395796,0.030861,-0.112841,0.935131,0.067614
2024-11-01,48.147764,49.467535,0.106572,-0.020038,0.920946,0.099968
2024-11-02,48.241675,49.56655,0.149745,0.024075,0.897546,0.12811


In [301]:
px.line(
    df_cohort,
    x="modeldate",
    y=["counterfactual", "oct_28_mover"],
    template="ggplot2",
    title="voteshare incumbent raw",
).update_layout(
    yaxis_title="voteshare_inc",
).add_annotation(
    x=df_cohort["modeldate"].min(),
    y=max(df_cohort["counterfactual"].max(), df_cohort["oct_28_mover"].max()),
    text=f"COHORT_1: {', '.join(COHORT_1)}<br>COHORT_2: {', '.join(COHORT_2)}",
    showarrow=False,
    align='left',
).show()
px.line(
    df_cohort,
    x="modeldate",
    y=["counterfactual_diff", "oct_28_mover_diff"],
    template="ggplot2",
    title="voteshare incumbent 1st diff",
).update_layout(
    yaxis_title="voteshare_inc diff",
).add_annotation(
    x=df_cohort["modeldate"].min(),
    y=max(df_cohort["counterfactual_diff"].max(), df_cohort["oct_28_mover_diff"].max()),
    text=f"COHORT_1: {', '.join(COHORT_1)}<br>COHORT_2: {', '.join(COHORT_2)}",
    showarrow=False,
    align='left',
).show()


In [302]:
px.line(
    df_cohort.filter(pl.col("corr").is_not_null()),
    x="modeldate",
    y="corr",
    template="plotly_dark",
    title=f"Rolling correlation of incumbent voteshare between swing state cohorts<br><sup>window={WINDOW_SIZE}</sup>",
    width=1000,
    height=600,
).update_layout(
    showlegend=False,
    xaxis_title=None,
    yaxis_title="correlation (1st diff)",
    plot_bgcolor="#171717",
    paper_bgcolor="#171717",
    xaxis_showgrid=False,
).add_annotation(
    x=-0.08,
    y=-0.15,
    xref="paper",
    yref="paper",
    text=f"COHORT_1: {', '.join(COHORT_1)}<br>COHORT_2: {', '.join(COHORT_2)}",
    showarrow=False,
    font=dict(
        size=10,
        color="grey"
    ),
    align='left',
).add_annotation(
    x=1.07,
    y=-0.15,
    xref="paper",
    yref="paper",
    text="<br>".join(
        (
            "Chart by Dominic Tarro | 𝕏 @dominictarro",
            "Source: Silver Bulletin 2024 presidential election forecast, Voteshare_Probabilities.xlsx",
        )
    ),
    align="right",
    showarrow=False,
    font=dict(
        size=10,
        color="grey"
    ),
    opacity=0.7
).show()

px.line(
    df_cohort.filter(pl.col("corr_diff").is_not_null()),
    x="modeldate",
    y="corr_diff",
    template="plotly_dark",
    title=f"Rolling correlation of incumbent voteshare between swing state cohorts<br><sup>window={WINDOW_SIZE} lag={DIFF_LAG}</sup>",
    width=1000,
    height=600,
).update_layout(
    showlegend=False,
    xaxis_title=None,
    yaxis_title="correlation (1st diff)",
    plot_bgcolor="#171717",
    paper_bgcolor="#171717",
    xaxis_showgrid=False,
).add_annotation(
    x=-0.08,
    y=-0.15,
    xref="paper",
    yref="paper",
    text=f"COHORT_1: {', '.join(COHORT_1)}<br>COHORT_2: {', '.join(COHORT_2)}",
    showarrow=False,
    font=dict(
        size=10,
        color="grey"
    ),
    align='left',
).add_annotation(
    x=1.07,
    y=-0.15,
    xref="paper",
    yref="paper",
    text="<br>".join(
        (
            "Chart by Dominic Tarro | 𝕏 @dominictarro",
            "Source: Silver Bulletin 2024 presidential election forecast, Voteshare_Probabilities.xlsx",
        )
    ),
    align="right",
    showarrow=False,
    font=dict(
        size=10,
        color="grey"
    ),
    opacity=0.7
).show()

In [303]:
poll = pl.read_excel("./Pres_Pollist.xlsx", engine="xlsx2csv", schema_overrides=dict(samplesize=pl.Float64, pct=pl.Float64))\
    .pipe(convert_modeldate_to_date)\
    .pipe(convert_modeldate_to_date, col="startdate")\
    .pipe(convert_modeldate_to_date, col="enddate")\
    .filter(pl.col("candidate_name").str.starts_with("Convention Bounce").not_())\
    .with_columns(
        pl.col("candidate_name").eq("Donald Trump").alias("is_trump"),
    )
poll

state,modeldate,candidate_name,startdate,enddate,pollster,pollster_rating_id,samplesize,population,weight,influence,pct,house_adjusted_pct,trend_and_house_adjusted_pct,tracking,poll_id,question_id,is_trump
str,date,str,date,date,str,i64,f64,str,f64,f64,f64,f64,f64,str,i64,i64,bool
"""Georgia""",2024-11-04,"""Kamala Harris""",2024-11-02,2024-11-03,"""InsiderAdvantage""",243,800.0,"""lv""",0.992316,0.992316,47.700001,48.28471,48.18829,null,89319,216169,false
"""Wisconsin""",2024-11-04,"""Kamala Harris""",2024-11-02,2024-11-03,"""Research Co.""",449,450.0,"""lv""",0.851658,0.851658,49.0,48.72274,48.731091,null,89330,216181,false
"""Michigan""",2024-11-04,"""Kamala Harris""",2024-11-02,2024-11-03,"""Research Co.""",449,450.0,"""lv""",0.851658,0.851658,49.0,48.524399,48.523842,null,89322,216173,false
"""California""",2024-11-04,"""Kamala Harris""",2024-11-02,2024-11-03,"""Research Co.""",449,450.0,"""lv""",0.851658,0.851658,64.0,63.042759,63.067558,null,89320,216171,false
"""Missouri""",2024-11-04,"""Kamala Harris""",2024-11-02,2024-11-03,"""Research Co.""",449,450.0,"""lv""",0.851658,0.851658,39.0,39.280392,39.284561,null,89324,216175,false
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""National""",2024-11-04,"""Donald Trump""",2024-07-01,2024-07-03,"""Data for Progress""",522,2067.0,"""lv""",1.084298,0.0,46.0,46.352669,47.28352,null,87366,202610,true
"""National""",2024-11-04,"""Donald Trump""",2024-06-28,2024-07-01,"""YouGov""",391,1176.0,"""rv""",1.084334,0.0,47.0,48.252861,49.207901,null,87298,201923,true
"""National""",2024-11-04,"""Donald Trump""",2024-06-28,2024-06-30,"""SSRS""",297,1274.0,"""rv""",0.947891,0.0,47.0,47.226521,48.090469,null,87294,201882,true


In [304]:
poll_swing = poll.filter(
    pl.col("population") == "lv",
    pl.col("state").is_in(SWING_STATES),
    pl.col("startdate") > datetime.date(2024, 8, 25),
).pivot(
    index=["state", "poll_id"],
    on="candidate_name",
    values="pct",
).rename(
    {
        "Donald Trump": "trump_pct",
        "Kamala Harris": "harris_pct",
    }
).with_columns(
    pl.col("harris_pct").sub(pl.col("trump_pct")).alias("harris_lead_pct"),
).join(
    poll.group_by("poll_id").first().select("poll_id", "pollster", "enddate", "samplesize", "influence"),
    on="poll_id",
).sort(
    "state", "enddate", "pollster",
)
poll_swing

state,poll_id,harris_pct,trump_pct,harris_lead_pct,pollster,enddate,samplesize,influence
str,i64,f64,f64,f64,str,date,f64,f64
"""Arizona""",87990,48.400002,48.599998,-0.199997,"""InsiderAdvantage""",2024-08-31,800.0,0.0
"""Arizona""",88069,47.400002,47.25,0.150002,"""TIPP Insights""",2024-09-05,949.0,0.0
"""Arizona""",88085,46.889999,49.049999,-2.16,"""Morning Consult""",2024-09-08,901.0,0.0
"""Arizona""",88203,46.0,46.200001,-0.200001,"""Data Orbital""",2024-09-09,550.0,0.0
"""Arizona""",88114,46.0,47.0,-1.0,"""Redfield & Wilton Strategies""",2024-09-09,765.0,0.0
…,…,…,…,…,…,…,…,…
"""Wisconsin""",89346,49.0,48.0,1.0,"""Florida Atlantic University/Ma…",2024-11-02,null,0.749424
"""Wisconsin""",89304,48.0,49.0,-1.0,"""InsiderAdvantage""",2024-11-02,800.0,0.79052
"""Wisconsin""",89276,48.5,46.0,2.5,"""The New York Times/Siena Colle…",2024-11-02,1305.0,1.513228


In [305]:
poll.filter(pl.col("pollster") == "Selzer & Co.")

state,modeldate,candidate_name,startdate,enddate,pollster,pollster_rating_id,samplesize,population,weight,influence,pct,house_adjusted_pct,trend_and_house_adjusted_pct,tracking,poll_id,question_id,is_trump
str,date,str,date,date,str,i64,f64,str,f64,f64,f64,f64,f64,str,i64,i64,bool
"""Iowa""",2024-11-04,"""Kamala Harris""",2024-10-28,2024-10-31,"""Selzer & Co.""",304,808.0,"""lv""",1.496203,1.433688,47.0,46.938271,46.93187,null,89241,215768,false
"""Iowa""",2024-11-04,"""Kamala Harris""",2024-09-08,2024-09-11,"""Selzer & Co.""",304,656.0,"""lv""",1.385807,0.247052,43.0,42.9389,42.622459,null,88179,209514,false
"""Iowa""",2024-11-04,"""Donald Trump""",2024-10-28,2024-10-31,"""Selzer & Co.""",304,808.0,"""lv""",1.496203,1.433688,44.0,44.94072,44.90353,null,89241,215768,true
"""Iowa""",2024-11-04,"""Donald Trump""",2024-09-08,2024-09-11,"""Selzer & Co.""",304,656.0,"""lv""",1.385807,0.247052,47.0,47.946571,48.840759,null,88179,209514,true


In [306]:
px.scatter(
    poll_swing.filter(
        pl.col("samplesize").is_not_null(),
        pl.col("enddate") > datetime.date(2024, 10, 1),
    ),
    x="samplesize",
    y="influence",
    # log_x=True,
    facet_col="state",
    hover_data=["pollster", "samplesize", "harris_pct", "trump_pct", "enddate"],
    color="harris_lead_pct",
    color_continuous_scale=px.colors.diverging.RdYlGn,
)